In [21]:
cd /ivi/ilps/personal/stan1/quac/quac_extractive

/ivi/ilps/personal/stan1/quac/quac_extractive


In [1]:
import pandas as pd
import json
from datasets import Dataset
from datasets.dataset_dict import DatasetDict

import torch
import torch.nn as nn
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, DataCollatorWithPadding
from transformers import BertTokenizerFast, LongformerTokenizerFast
from transformers import BertForQuestionAnswering, LongformerForQuestionAnswering

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


device(type='cuda')

In [14]:
def predictTheAnswer(text,question,model,max_length):
  inputs = tokenizer.encode_plus(question, text, return_tensors='pt',max_length=max_length, truncation=True).to(device)

  outputs = model(**inputs)
  answer_start = torch.argmax(outputs[0])  
  answer_end = torch.argmax(outputs[1]) + 1 

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
  return answer

def prediction(contents,model,max_length):
  predAnswers={}
  for data in contents['data']:
    for txt in data['paragraphs']: 
      text = txt['context']
      for qa in txt['qas']:
        qid = qa['id']
        question = qa['question']
        pred = predictTheAnswer(text,question,model,max_length)

        # empty '' means CANNOTANSWER
        if pred!='':
          predAnswers[qid]=pred
        else:
          predAnswers[qid]='CANNOTANSWER'
  return predAnswers

def ready_for_eval(pred_file_dir, processed_pred_dir):
    #load and post-process
    with open(pred_file_dir,'r') as json_file:
        data = json.load(json_file)
    data = post_processing(data)

    #store
    output_file = open(processed_pred_dir, 'w', encoding='utf-8')
    for dic in data:
       json.dump(dic, output_file) 
       output_file.write("\n")
    
    print('done!')
    return
    
def post_processing(data):
    template = {'best_span_str': [], 'qid':[], 'yesno': [], 'followup': []}
    diag_l = []
    prev_diag_ind =None

    for diag in list(data.keys()):
        
        # [:38] is the dialog index
        cur_diag_ind = diag[:38]

        # remove additional spaces
        best_span_str = data[diag]
        if best_span_str[0]==' ':
            best_span_str = best_span_str[1:]
        if best_span_str[-1]==' ':
            best_span_str = best_span_str[:-1]

        #still in one dialog
        if cur_diag_ind==prev_diag_ind:
            template['best_span_str'].append(best_span_str)
            template['qid'].append(diag)
            template['yesno'].append('')
            template['followup'].append('')

            prev_diag_ind = cur_diag_ind

        #now is different dialog
        else:
            diag_l.append(template)
            template = {'best_span_str': [], 'qid':[], 'yesno': [], 'followup': []}
            template['best_span_str'].append(best_span_str)
            template['qid'].append(diag)
            template['yesno'].append('')
            template['followup'].append('')

            prev_diag_ind = cur_diag_ind

    #first is empty template
    diag_l = diag_l[1:]
    return diag_l    

In [ ]:
#load trained model & tokenizer

checkpoint_dir = '/ivi/ilps/personal/stan1/quac/quac_extractive/bert-base-uncased-finetuned-quac/checkpoint-5000'
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', cache_dir="hf_models/bert_base//")
model = BertForQuestionAnswering.from_pretrained(checkpoint_dir)
model.cuda()

#checkpoint_dir = '/ivi/ilps/personal/stan1/quac/quac_extractive/longformer-base-uncased-finetuned-quac/checkpoint-12000'
#tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096', cache_dir="hf_models/longformer/")
#model = LongformerForQuestionAnswering.from_pretrained(checkpoint_dir)
#model.cuda()

In [8]:
val = pd.read_json("data/quac_dev.json")
val = val[['data']]
predAnswers=prediction(val, model, max_length=512)

In [16]:
with open('/ivi/ilps/personal/stan1/quac/quac_extractive/bert-base-uncased-finetuned-quac/results.txt', 'w') as convert_file:
     convert_file.write(json.dumps(predAnswers))

In [26]:
#post-processing
ready_for_eval('bert-base-uncased-finetuned-quac/results.txt', 'bert-base-uncased-finetuned-quac/processed_results_bert_base.txt')
#ready_for_eval('longformer-base-uncased-finetuned-quac/results.txt', 'longformer-base-uncased-finetuned-quac/processed_results_longformer_base.txt')

done!


In [27]:
#official evaluation
#!python toolbox/scorer.py --val_file data/val_v0.2.json --model_output bert-base-uncased-finetuned-quac/processed_results_bert_base.txt --o results/bert_base_eval_result.json --verbose
!python toolbox/scorer.py --val_file data/val_v0.2.json --model_output longformer-base-uncased-finetuned-quac/processed_results_longformer_base.txt --o results/longformer_base_eval_result.json --verbose

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--------------------
In May 1983, she married Nikos Karvelas,
['In May 1983, she married Nikos Karvelas,', 'In May 1983, she married Nikos Karvelas, a composer,', 'In May 1983, she married Nikos Karvelas,', 'In May 1983, she married Nikos Karvelas, a composer,']
1.0
--------------------
--------------------
in November she gave birth to her daughter Sofia.
['she gave birth to her daughter Sofia.', 'in 1975 and in November she gave birth to her daughter Sofia.', 'her daughter Sofia.', 'in November she gave birth to her daughter Sofia.']
0.96875
--------------------
--------------------
CANNOTANSWER
['CANNOTANSWER']
1.0
--------------------
--------------------
After th

### deprecated 

In [26]:
#deprecated 
!python toolbox/evaluate-v2.0.py data/quac_dev.json /ivi/ilps/personal/stan1/quac/quac_extractive/longformer-base-uncased-finetuned-quac/results.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "exact": 25.972259994560783,
  "f1": 39.60979909293544,
  "total": 7354,
  "HasAns_exact": 25.972259994560783,
  "HasAns_f1": 39.60979909293544,
  "HasAns_total": 7354
}


In [12]:
!python toolbox/evaluate-v2.0.py data/quac_dev.json /ivi/ilps/personal/stan1/quac/quac_extractive/longformer-base-uncased-finetuned-quac/results.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "exact": 24.66684797389176,
  "f1": 38.30438707226639,
  "total": 7354,
  "HasAns_exact": 24.66684797389176,
  "HasAns_f1": 38.30438707226639,
  "HasAns_total": 7354
}


In [14]:
!python toolbox/evaluate-v2.0.py data/quac_dev.json /ivi/ilps/personal/stan1/quac/quac_extractive/bert-base-uncased-finetuned-quac/results.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "exact": 12.850149578460702,
  "f1": 27.869870930033535,
  "total": 7354,
  "HasAns_exact": 12.850149578460702,
  "HasAns_f1": 27.869870930033535,
  "HasAns_total": 7354
}


In [13]:
!python toolbox/evaluate-v2.0.py data/quac_dev.json /ivi/ilps/personal/stan1/quac/quac_extractive/bert-base-cased-finetuned-quac/results.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "exact": 13.27168887680174,
  "f1": 27.87972608128516,
  "total": 7354,
  "HasAns_exact": 13.27168887680174,
  "HasAns_f1": 27.87972608128516,
  "HasAns_total": 7354
}
